In [1]:
%reset -f

In [2]:
import spacy
from spacy.tokens import Doc
import PyPDF2
import pandas as pd

In [3]:
# Creating a pdf reader object
reader = PyPDF2.PdfReader('fraport/Annual Report 2019.pdf')
input_text = ''

for i in reader.pages:
    input_text += i.extract_text()

input_text = input_text.replace('\n', ' ')
# input_text = 'At Bosch, diversity is a fundamental pillar. We ensured a diverse workforce by taking necessary measures in the past. Still, The company intends to increase workforce diversity by 40%. It would enable a healthy and balanced workforce that would be warm and welcoming for people from diverse backgrounds.'

In [4]:
# First stage - Coreference Resolution
coref_model = spacy.load('en_coreference_web_trf')
coref_model.max_length = 2000000
doc = coref_model(input_text)

del reader, input_text

def resolve_references(doc):
    token_mention_mapper = {}
    output_string = ''
    clusters = [val for key, val in doc.spans.items() if key.startswith('coref_cluster')]

    for cluster in clusters:
        first_mention = cluster[0]

        for mention_span in list(cluster)[1:]:
            if len(mention_span) == 0:
                continue
            else:
                token_mention_mapper[mention_span[0].idx] = first_mention.text + mention_span[0].whitespace_
                for token in mention_span[1:]:
                    token_mention_mapper[token.idx] = ''

    for token in doc:
        if token.idx in token_mention_mapper:
            output_string += token_mention_mapper[token.idx]
        else:
            output_string += token.text + token.whitespace_

    return output_string

resolved_string = resolve_references(doc)

del coref_model, doc

spacy_core = spacy.load('en_core_web_trf')
doc = spacy_core(resolved_string)

# Second and third stage - NER and POS tagging
action_statements = list()
flag = False
list_of_sent = list(doc.sents)
for sent in list_of_sent:
    flag = False
    sent = list(sent)
    for word in sent:
        if word.ent_type_ == 'ORG':
            location = sent.index(word)
            if (location+1 != len(sent)) and sent[location+1].tag_ in ('VB', 'VBG', 'VBP', 'VBZ', 'MD', 'VV', 'VP', 'VERB', 'VAFIN', 'VMFIN', 'VVFIN', 'VE'):
                flag = True
                break
    if flag:
        str_sent = list(map(lambda x: str(x), sent))
        action_statements.append(' '.join(str_sent).replace(' - ', ' '))

del spacy_core, doc

Token indices sequence length is longer than the specified maximum sequence length for this model (1242 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 340837402248 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
df = pd.Series(action_statements)
df.to_csv('fraport_2019.csv', index=False)